In [37]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG, BTCUSD

import pandas as pd
import pandas_ta as ta

In [68]:
from backtesting import Backtest, Strategy
import pandas_ta as ta

class MACDBollingerBandsMeanReversion(Strategy):    
    '''
    An indicators + mean reversion trading strategy based on
    https://www.youtube.com/watch?v=qShed6dyrQY.
    Good for prices in range-bound markets.

    NOTE: The video uses Bollinger bands with a length of 200.
    Unsure why they choose this length; sticking to the default 20.

    NOTE: The video also has a trend-following strategy that we could try.
    Not implemented yet.
    '''
        
    def init(self):
        self.macd_line, self.macd_hist, self.macd_signal = self.I(
            ta.macd, self.data.Close.s, fast=12, slow=26, signal=9
        )

        def bb_lower(close):
            return ta.bbands(close, length=20, std=2)["BBL_20_2.0"]
        
        def bb_upper(close):
            return ta.bbands(close, length=20, std=2)["BBU_20_2.0"]
        
        self.lower_band = self.I(bb_lower, self.data.Close.s)
        self.upper_band = self.I(bb_upper, self.data.Close.s)
    
    def next(self):
        if len(self.data.Close) < 2:
            return
        
        current_close = self.data.Close[-1]
        previous_close = self.data.Close[-2]
        
        current_lower = self.lower_band[-1]
        previous_lower = self.lower_band[-2]
        current_upper = self.upper_band[-1]
        previous_upper = self.upper_band[-2]
        
        current_macd_hist = self.macd_hist[-1]
        previous_macd_hist = self.macd_hist[-2]
        
        if (
            previous_close < current_close and
            previous_macd_hist < current_macd_hist and
            previous_close < previous_lower and
            current_close > current_lower
        ):
            self.buy()
        
        elif (
            previous_close > current_close and
            previous_macd_hist > current_macd_hist and
            previous_close > previous_upper and
            current_close < current_upper
        ):
            self.sell()


In [69]:
bt = Backtest(GOOG, MACDBollingerBandsMeanReversion,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()

GridPlot(id='p4276', ...)